In [41]:
!pip install gluonnlp tqdm
!pip install mxnet
!pip install sentencepiece==0.1.91
!pip install transformers==4.8.2
!pip install torch==1.8.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 804.1 MB 2.5 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.8.1 which is incompatible.
torch

In [1]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-ikcv3v_9/kobert-tokenizer_36487bbb68e148a385b0c070d5664e7a
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-ikcv3v_9/kobert-tokenizer_36487bbb68e148a385b0c070d5664e7a
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-rav04yi3
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-rav04yi3


In [2]:
from kobert.pytorch_kobert import get_pytorch_kobert_model
from kobert_tokenizer import KoBERTTokenizer

tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

In [4]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel

device = torch.device("cuda:0")

In [5]:
data = pd.read_csv("/content/sentiment_data.csv")
data.head()

,Unnamed: 0,Sentence,Emotion,From
0,0,언니 동생으로 부르는게 맞는 일인가요..??,공포,단발성 대화
1,1,그냥 내 느낌일뿐겠지?,공포,단발성 대화
2,2,아직너무초기라서 그런거죠?,공포,단발성 대화
3,3,유치원버스 사고 낫다던데,공포,단발성 대화
4,4,근데 원래이런거맞나요,공포,단발성 대화


In [6]:
data.loc[data['Emotion'] == "공포", "Emotion"] = 0
data.loc[data['Emotion'] == "놀람", "Emotion"] = 1
data.loc[data['Emotion'] == "분노", "Emotion"] = 2
data.loc[data['Emotion'] == "슬픔", "Emotion"] = 3
data.loc[data['Emotion'] == "중립", "Emotion"] = 4
data.loc[data['Emotion'] == "행복", "Emotion"] = 5
data.loc[data['Emotion'] == "혐오", "Emotion"] = 6

In [7]:
data_list = []
for ques, label in zip(data['Sentence'], data['Emotion']):
    data = []
    data.append(ques)
    data.append(str(label))
    data_list.append(data)

In [23]:
class BERTDataset(Dataset):
  def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len, pad, pair):
    transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)
    
    self.sentences = [transform([i[sent_idx]]) for i in dataset]
    self.labels = [np.int32(i[label_idx]) for i in dataset]
  
  def __getitem__(self, i):
    return (self.sentences[i] + (self.labels[i], ))
  
  def __len__(self):
    return (len(self.labels))

In [24]:
# setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

In [25]:
# train 데이터, test 데이터로 나누기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=34)

In [26]:
tok = tokenizer.tokenize
data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, vocab, max_len, True, False)

In [34]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [35]:
class BERTClassifier(nn.Module):
  def __init__(self,
               bert,
               hidden_size = 768,
               num_classes = 7,  # 클래스 수 조정
               dr_rate = None,
               params = None):
    super(BERTClassifier, self).__init__()
    self.bert = bert
    self.dr_rate = dr_rate

    self.classifier = nn.Linear(hidden_size, num_classes)
    if dr_rate:
      self.dropout = nn.Dropout(p=dr_rate)

  def gen_attention_mask(self, token_ids, valid_length):
    attention_mask = torch.zeros_like(token_ids)
    for i, v in enumerate(valid_length):
      attention_mask[i][:v] = 1
    return attention_mask.float()
  
  def forward(self, token_ids, valid_length, segment_ids):
    attention_mask = self.gen_attention_mask(token_ids, valid_length)
    _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device), return_dict=False)
    if self.dr_rate:
      out = self.dropout(pooler)
    return self.classifier(out)

In [36]:
# BERT 모델 불러오기
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

# optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [37]:
# 정확도 측정을 위한 함수 정의
def calc_accuracy(X, Y):
  max_vals, max_indices = torch.max(X, 1)
  train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
  return train_acc


In [38]:
train_history=[]
test_history=[]
loss_history=[]
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
         
        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/1323 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 1.871563196182251 train acc 0.1875
epoch 1 batch id 201 loss 1.0865551233291626 train acc 0.5417444029850746
epoch 1 batch id 401 loss 0.9919701814651489 train acc 0.6200903990024937
epoch 1 batch id 601 loss 0.9567520022392273 train acc 0.6500363976705491
epoch 1 batch id 801 loss 0.8530606627464294 train acc 0.6694366416978776
epoch 1 batch id 1001 loss 0.862583339214325 train acc 0.683425949050949
epoch 1 batch id 1201 loss 0.7168813943862915 train acc 0.6915981473771857
epoch 1 train acc 0.6955057406104204


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 test acc 0.6947194189111011


  0%|          | 0/1323 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.28694581985473633 train acc 0.90625
epoch 5 batch id 201 loss 0.30550888180732727 train acc 0.8893034825870647
epoch 5 batch id 401 loss 0.29968202114105225 train acc 0.894443578553616
epoch 5 batch id 601 loss 0.36762934923171997 train acc 0.9007643510815307
epoch 5 batch id 801 loss 0.1956634223461151 train acc 0.9061719725343321
epoch 5 batch id 1001 loss 0.2388172745704651 train acc 0.9106362387612388
epoch 5 batch id 1201 loss 0.28836962580680847 train acc 0.9117141965029142
epoch 5 train acc 0.9125151497380563


  0%|          | 0/331 [00:00<?, ?it/s]

epoch 5 test acc 0.6958222182940155


In [39]:
def predict(predict_sentence):
  data = [predict_sentence, '0']
  dataset_another = [data]

  another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
  test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

  model.eval()

  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length = valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)

    test_eval = []

    for i in out:
      logits = i
      logits = logits.detach().cpu().numpy()

      if np.argmax(logits) == 0:
        test_eval.append("공포가")
      elif np.argmax(logits) == 1:
        test_eval.append("놀람이")
      elif np.argmax(logits) == 2:
          test_eval.append("분노가")
      elif np.argmax(logits) == 3:
          test_eval.append("슬픔이")
      elif np.argmax(logits) == 4:
          test_eval.append("중립이")
      elif np.argmax(logits) == 5:
          test_eval.append("행복이")
      elif np.argmax(logits) == 6:
          test_eval.append("혐오가")

  print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

In [41]:
end = 1
while end == 1:
  sentence = input("입력: ")
  if sentence == "0":
    break
  predict(sentence)
  print("\n")

입력: 오늘 맛있는거 먹었음


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


>> 입력하신 내용에서 행복이 느껴집니다.


입력: 기분이 안좋아요.
>> 입력하신 내용에서 혐오가 느껴집니다.


입력: 나는 오이가 싫다.
>> 입력하신 내용에서 혐오가 느껴집니다.


입력: 0
